## Сравнение различных стратегий в дилемме заключённого

##### Класс экперимент и определение стратегий

In [1]:
import numpy as np
from typing import Callable, Tuple, List, Dict

class Experiment:
    
    def __init__(self, strategy_a: Callable, strategy_b: Callable, num_rounds: int = 200):

        self.strategy_a = strategy_a
        self.strategy_b = strategy_b
        self.num_rounds = num_rounds
        
        self.payoff_matrix = {
            (0, 0): (3, 3),  
            (0, 1): (0, 5),  
            (1, 0): (5, 0),  
            (1, 1): (1, 1)   
        }
        
        self.history_a = []  
        self.history_b = []  
        self.scores_a = []   
        self.scores_b = []   
        
    def play_round(self, round_num: int) -> Tuple[int, int]:
        prev_moves_a = self.history_a[:round_num]
        prev_moves_b = self.history_b[:round_num]
        
        move_a = self.strategy_a(prev_moves_a, prev_moves_b, round_num)
        move_b = self.strategy_b(prev_moves_b, prev_moves_a, round_num)
        
        return move_a, move_b
    
    def run_experiment(self) -> Dict:
        self.history_a = []
        self.history_b = []
        self.scores_a = []
        self.scores_b = []
        
        for round_num in range(self.num_rounds):
            move_a, move_b = self.play_round(round_num)
            
            self.history_a.append(move_a)
            self.history_b.append(move_b)
            
            score_a, score_b = self.payoff_matrix[(move_a, move_b)]
            self.scores_a.append(score_a)
            self.scores_b.append(score_b)

        return self.analyze_results()
    
    def analyze_results(self) -> Dict:
        total_score_a = sum(self.scores_a)
        total_score_b = sum(self.scores_b)

        dominant_series_a = self.find_dominant_series(self.scores_a, self.scores_b)
        dominant_series_b = self.find_dominant_series(self.scores_b, self.scores_a)
        
        return {
            'total_scores': (total_score_a, total_score_b),
            'dominant_series': (dominant_series_a, dominant_series_b),
            'moves_a': self.history_a,
            'moves_b': self.history_b,
            'scores_a': self.scores_a,
            'scores_b': self.scores_b
        }
    
    def find_dominant_series(self, scores1: List[int], scores2: List[int]) -> int:
        max_series = 0
        current_series = 0
        
        for s1, s2 in zip(scores1, scores2):
            if s1 == 5 and s2 == 0:
                current_series += 1
                max_series = max(max_series, current_series)
            else:
                current_series = 0
        
        return max_series

def strategy_alex(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 1

def strategy_bob(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 0

def strategy_clara(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    return opp_prev[-1] if opp_prev else 0

def strategy_denis(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    return 1 - opp_prev[-1] if opp_prev else 0

def strategy_emma(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 1 if (round_num + 1) % 20 == 0 else 0

def strategy_frida(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if not opp_prev:
        return 0
    return 0 if all(move == 0 for move in opp_prev) else 1

def strategy_merciful_clara(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    
    if (round_num + 1) % 10 == 0:
        return 0

    return opp_prev[-1] if opp_prev else 0

def compare_all_strategies():
    strategies = {
        'Alex': strategy_alex,
        'Bob': strategy_bob,
        'Clara': strategy_clara,
        'Denis': strategy_denis,
        'Emma': strategy_emma,
        'Frida': strategy_frida,
        'merciful_Clara': strategy_merciful_clara
    }
    
    results = {}
    strategy_names = list(strategies.keys())
    
    for i, name1 in enumerate(strategy_names):
        for j, name2 in enumerate(strategy_names):
            if i <= j:
                experiment = Experiment(
                    strategies[name1], 
                    strategies[name2]
                )
                result = experiment.run_experiment()
                
                key = f"{name1} vs {name2}"
                results[key] = result
    
    return results, strategies


##### Сравнение отдельных стратегий

In [2]:

experiment1 = Experiment(strategy_alex, strategy_bob)
result = experiment1.run_experiment()
    
print("Результаты Alex vs Bob:")
print(f"Общие очки: Alex={result['total_scores'][0]}, Bob={result['total_scores'][1]}")
print(f"Макс. доминирующая серия: Alex={result['dominant_series'][0]}, Bob={result['dominant_series'][1]}")

experiment2 = Experiment(strategy_clara, strategy_denis)
result = experiment2.run_experiment()
    
print("Результаты Clara vs Denis:")
print(f"Общие очки: Clara={result['total_scores'][0]}, Denis={result['total_scores'][1]}")
print(f"Макс. доминирующая серия: Clara={result['dominant_series'][0]}, Denis={result['dominant_series'][1]}")

experiment3 = Experiment(strategy_clara, strategy_merciful_clara)
result = experiment3.run_experiment()
    
print("Результаты Clara vs merciful Clara:")
print(f"Общие очки: Clara={result['total_scores'][0]}, merciful Clara={result['total_scores'][1]}")
print(f"Макс. доминирующая серия: Clara={result['dominant_series'][0]}, merciful Clara={result['dominant_series'][1]}")

Результаты Alex vs Bob:
Общие очки: Alex=1000, Bob=0
Макс. доминирующая серия: Alex=200, Bob=0
Результаты Clara vs Denis:
Общие очки: Clara=450, Denis=450
Макс. доминирующая серия: Clara=1, Denis=1
Результаты Clara vs merciful Clara:
Общие очки: Clara=600, merciful Clara=600
Макс. доминирующая серия: Clara=0, merciful Clara=0


##### Таблица сравнения всех стратегий

In [9]:
all_results, strategies = compare_all_strategies()
    
print("\nТАБЛИЦА ОБЩИХ ОЧКОВ:")
header = "Стратегия1 vs Стратегия2            | Очки1 | Очки2"
print(header)
print("-" * len(header))
    
for match, result in all_results.items():
    scores = result['total_scores']
    print(f"{match:35} | {scores[0]:5} | {scores[1]:5}")
    
print("\nТАБЛИЦА ДОМИНИРУЮЩИХ СЕРИЙ:")
header = "Стратегия1 vs Стратегия2            | Серия1 | Серия2"
print(header)
print("-" * len(header))
    
for match, result in all_results.items():
    series = result['dominant_series']
    print(f"{match:35} | {series[0]:6} | {series[1]:6}")


ТАБЛИЦА ОБЩИХ ОЧКОВ:
Стратегия1 vs Стратегия2            | Очки1 | Очки2
---------------------------------------------------
Alex vs Alex                        |   200 |   200
Alex vs Bob                         |  1000 |     0
Alex vs Clara                       |   204 |   199
Alex vs Denis                       |  1000 |     0
Alex vs Emma                        |   960 |    10
Alex vs Frida                       |   204 |   199
Alex vs merciful_Clara              |   284 |   179
Bob vs Bob                          |   600 |   600
Bob vs Clara                        |   600 |   600
Bob vs Denis                        |     3 |   998
Bob vs Emma                         |   570 |   620
Bob vs Frida                        |   600 |   600
Bob vs merciful_Clara               |   600 |   600
Clara vs Clara                      |   600 |   600
Clara vs Denis                      |   450 |   450
Clara vs Emma                       |   588 |   593
Clara vs Frida                      |   60